# 실험 설명

저는 데이터를 구글 드라이브에 넣고 진행했습니다

아래 경로를 체크해서 오류가 나지않게 조심하시길 바랍니다

## 학습 소요시간

아래 모델 기준 학습에 5 epoch에 30분정도 소요되었습니다


In [1]:

# 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 데이터 load 및 압축 해제

import os
import zipfile

# 데이터 루트 경로 - 미리 다운받아둔 데이터로 실험 진행
data_path = '/content/drive/MyDrive/Colab Notebooks/데싸/딥러닝/data'
lyrics_path = os.path.join(data_path, '작사가')

# 압축 파일 목록
zip_files = {
    'lyrics.zip': 'lyrics'
}

for zip_name, folder_name in zip_files.items():
    zip_path = os.path.join(lyrics_path, zip_name)           # zip 파일 전체 경로
    extract_dir = os.path.join(lyrics_path, folder_name)     # 압축 해제될 폴더

    # 폴더 없으면 생성
    os.makedirs(extract_dir, exist_ok=True)

    print(f"🔍 압축 해제 중: {zip_name} → {extract_dir}")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

print("✅ 모든 압축 해제 완료!")

🔍 압축 해제 중: lyrics.zip → /content/drive/MyDrive/Colab Notebooks/데싸/딥러닝/data/작사가/lyrics
✅ 모든 압축 해제 완료!


In [3]:
import glob #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import os

lyrics_file_path = '/content/drive/MyDrive/Colab Notebooks/데싸/딥러닝/data/작사가/lyrics/lyrics/*'

txt_list = glob.glob(lyrics_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당


In [4]:
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


In [5]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

corpus = list(map(preprocess_sentence, raw_corpus))

In [6]:
import numpy as np
import tensorflow as tf

# 토크나이저 함수로 Tensor 변환

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,
        filters=' ',
        oov_token="<unk>"
    )

    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 306  28 ...   0   0   0]
 [  2 221  13 ...   0   0   0]
 [  2  24  17 ...   0   0   0]
 ...
 [  2  23  77 ...   0   0   0]
 [  2  42  26 ...   0   0   0]
 [  2  23  77 ...   0   0   0]] <keras.src.legacy.preprocessing.text.Tokenizer object at 0x7a75ae11eb70>


In [7]:
from sklearn.model_selection import train_test_split

enc_inputs = tensor[:, :-1]
dec_targets = tensor[:, 1:]

# 20%를 평가 데이터로 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(
    enc_inputs,
    dec_targets,
    test_size=0.2,
    random_state=42,  # 재현성 확보 위해 시드 고정 (선택 사항)
    shuffle=True      # 데이터 섞기
)

In [8]:
enc_train

array([[   4, 2118,    4, ...,  137,    4,  137],
       [   2, 1264,    4, ...,   42,  309,    3],
       [   2,   49,    4, ...,    0,    0,    0],
       ...,
       [   2,   39,    4, ...,    0,    0,    0],
       [   2,  163, 5658, ...,    0,    0,    0],
       [   2,    8,   72, ...,    0,    0,    0]], dtype=int32)

In [9]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 64
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
val_dataset = val_dataset.shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
print(val_dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(64, 14), dtype=tf.int32, name=None), TensorSpec(shape=(64, 14), dtype=tf.int32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(64, 14), dtype=tf.int32, name=None), TensorSpec(shape=(64, 14), dtype=tf.int32, name=None))>


In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다.
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)

        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!
embedding_size = 256 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

In [11]:
optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저이다. 자세한 내용은 차차 배운다.
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다.
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
history = model.fit(
    dataset,
    validation_data=val_dataset,
    epochs=5
)

Epoch 1/5
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 378s 158ms/step - loss: 3.4578 - val_loss: 2.8779
Epoch 2/5
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 371s 158ms/step - loss: 2.7436 - val_loss: 2.6705
Epoch 3/5
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 372s 159ms/step - loss: 2.4629 - val_loss: 2.5409
Epoch 4/5
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 372s 159ms/step - loss: 2.2054 - val_loss: 2.4517
Epoch 5/5
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 371s 159ms/step - loss: 1.9688 - val_loss: 2.4007


In [12]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor)
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        # 3
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [23]:
generate_text(model, tokenizer, init_sentence="<start> start ", max_len=30)

'<start> start to get up to the sky <end> '